# Chapter 2: Graph Theory

Companion notebook for [Network Science](http://networksciencebook.com/) by Albert-László Barabási.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/grogs84/networkscience/blob/main/notebooks/Chapter2.ipynb)

---

## Topics Covered

- **2.3** Degree, Average Degree, Degree Distribution
- **2.4** Adjacency Matrix
- **2.7** Bipartite Graphs and Projections
- **2.8** Paths and Distances
- **2.9** Clustering Coefficient

## Setup

Run this cell to install required packages (only needed on Google Colab).

In [ ]:
# Install dependencies (for Google Colab)
import sys
if 'google.colab' in sys.modules:
    !pip install -q rustworkx matplotlib numpy

In [ ]:
# Import libraries
import rustworkx as rx
from rustworkx.visualization import mpl_draw
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
from itertools import combinations
import math

# Visualization settings
NS_PURPLE = '#8e44ad'
NS_GREEN = '#2ecc71'
NS_ORANGE = '#FF9800'

plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['figure.dpi'] = 100

---
## 2.3 Degree, Average Degree, Degree Distribution

### Key Definitions

| Term | Formula | Description |
|------|---------|-------------|
| **Degree** ($k_i$) | — | Number of edges connected to node $i$ |
| **Total Edges** | $L = \frac{1}{2}\sum_{i=1}^{N} k_i$ | Sum of all degrees divided by 2 |
| **Average Degree** | $\langle k \rangle = \frac{1}{N}\sum_{i=1}^{N} k_i = \frac{2L}{N}$ | Mean degree across all nodes |
| **Degree Distribution** | $p_k = \frac{N_k}{N}$ | Fraction of nodes with degree $k$ |

In [ ]:
# Create a simple graph
G = rx.PyGraph()
G.add_nodes_from([0, 1, 2, 3])
G.add_edges_from_no_data([(0, 1), (0, 2), (0, 3), (1, 2)])

# Calculate degree metrics
degrees = [G.degree(n) for n in G.node_indexes()]
avg_degree = sum(degrees) / len(degrees)

print(f"Degrees: {degrees}")
print(f"Average degree ⟨k⟩ = {avg_degree}")

# Visualize
mpl_draw(G, node_color=NS_PURPLE, edge_color=NS_GREEN, 
         with_labels=True, node_size=500, font_color='white')
plt.title("Simple Graph")
plt.show()

### Degree Distribution

The degree distribution $p_k$ gives the probability that a randomly selected node has degree $k$.

In [ ]:
def degree_distribution(G):
    """Calculate the degree distribution of a graph."""
    degrees = [G.degree(n) for n in G.node_indexes()]
    counts = Counter(degrees)
    N = len(degrees)
    return {k: count / N for k, count in sorted(counts.items())}

p_k = degree_distribution(G)
print(f"Degree distribution: {p_k}")

---
## 2.4 Adjacency Matrix

For mathematical purposes we often represent a network through its **adjacency matrix**. The adjacency matrix of a network with $N$ nodes has $N$ rows and $N$ columns, with elements:

- $A_{ij} = 1$ if there is a link pointing from node $j$ to node $i$
- $A_{ij} = 0$ if nodes $i$ and $j$ are not connected

In [ ]:
# Define adjacency matrix
A = np.array([
    [0, 1, 1, 0],
    [1, 0, 1, 1],
    [1, 1, 0, 0],
    [0, 1, 0, 0]
], dtype=float)

print(f"Adjacency Matrix Shape: {A.shape}\n")
print(A)

# Create graph from adjacency matrix
G = rx.PyGraph.from_adjacency_matrix(A)
mpl_draw(G, node_color=NS_PURPLE, edge_color=NS_GREEN, 
         with_labels=True, node_size=500, font_color='white')
plt.title("Graph from Adjacency Matrix")
plt.show()

### Degree from Adjacency Matrix

**Undirected graphs**: The degree of node $i$ can be computed by summing row or column $i$:

$$k_i = \sum_{j=1}^{N} A_{ij} = \sum_{j=1}^{N} A_{ji}$$

In [ ]:
# Degree for each node (sum along rows or columns)
ki = np.sum(A, axis=1)
print(f"Degree for nodes: {ki}")

# For undirected graphs, row and column sums are equal
assert np.all(np.sum(A, axis=1) == np.sum(A, axis=0))

**Directed graphs**: In-degree and out-degree are computed separately:

$$k_i^{in} = \sum_{j=1}^{N} A_{ij} \qquad k_i^{out} = \sum_{j=1}^{N} A_{ji}$$

In [ ]:
# Create a random directed graph
p, n = 0.3, 4
rng = np.random.default_rng(123)
A_dir = (rng.random((n, n)) < p).astype(float)
np.fill_diagonal(A_dir, 0)

G_dir = rx.PyDiGraph.from_adjacency_matrix(A_dir)
mpl_draw(G_dir, node_color=NS_PURPLE, edge_color=NS_GREEN, 
         with_labels=True, node_size=500, font_color='white')
plt.title("Directed Graph")
plt.show()

# In-degree (sum columns) and Out-degree (sum rows)
in_degrees = np.sum(A_dir, axis=0)
out_degrees = np.sum(A_dir, axis=1)

for i, (k_in, k_out) in enumerate(zip(in_degrees, out_degrees)):
    print(f"Node {i}: In-degree = {k_in}, Out-degree = {k_out}")

### Trace Tricks

Matrix powers of the adjacency matrix reveal important network properties:

| Expression | Meaning |
|------------|--------|
| $A^2_{ii}$ | Number of walks of length 2 starting and ending at node $i$ |
| $A^N_{ii}$ | Number of walks of length $N$ starting and ending at node $i$ |
| $\text{Tr}(A^3) / 6$ | Number of triangles in an undirected graph |

In [ ]:
# Create a graph with triangles
A = np.array([
    [0, 1, 1, 1],
    [1, 0, 1, 0],
    [1, 1, 0, 0],
    [1, 0, 0, 0]
], dtype=float)

G = rx.PyGraph.from_adjacency_matrix(A)
mpl_draw(G, node_color=NS_PURPLE, edge_color=NS_GREEN, 
         with_labels=True, node_size=500, font_color='white')
plt.title("Graph with Triangle")
plt.show()

In [ ]:
# Walks of length 2 for each node (diagonal of A²)
A2 = A @ A
walks_2 = np.diag(A2)
print(f"Walks of length 2: {walks_2}")

# Count triangles in undirected graph
A3 = np.linalg.matrix_power(A, 3)
num_triangles = int(np.trace(A3) // 6)
print(f"Number of triangles: {num_triangles}")

In [ ]:
# Walks of length N for each node
length = 3
for node, walks in zip(G.node_indexes(), np.diagonal(np.linalg.matrix_power(A, length))):
    print(f"Node {node} has {int(walks)} walks of length {length}")

---
## 2.7 Bipartite Graphs

### Key Definitions

| Term | Definition |
|------|------------|
| **Bipartite Graph** | A graph whose nodes can be divided into two disjoint sets $U$ and $V$ such that every edge connects a node in $U$ to a node in $V$ |
| **Two-Coloring** | A valid assignment of two colors to nodes where no adjacent nodes share a color |
| **Projection** | A new graph containing only nodes from one set, with edges connecting nodes that share a common neighbor |

In [ ]:
# Create a bipartite graph
G_bip = rx.PyGraph()
# Set U: nodes 0, 1, 2
# Set V: nodes 3, 4, 5
G_bip.add_nodes_from(range(6))
G_bip.add_edges_from_no_data([(0, 3), (0, 4), (1, 3), (1, 5), (2, 4), (2, 5)])

# Visualize with two-coloring
colors = [NS_PURPLE if n < 3 else NS_GREEN for n in range(6)]
pos = {0: (0, 2), 1: (0, 1), 2: (0, 0), 3: (2, 2), 4: (2, 1), 5: (2, 0)}
mpl_draw(G_bip, node_color=colors, edge_color='gray', pos=pos,
         with_labels=True, node_size=500, font_color='white')
plt.title("Bipartite Graph")
plt.show()

### Bipartite Coloring

In [ ]:
def bipartite_colors(G):
    """
    Return the two-coloring of a bipartite graph.
    Returns (U, V) where U and V are sets of node indices.
    """
    colors = rx.graph_two_color(G)
    U = {n for n, c in colors.items() if c == 0}
    V = {n for n, c in colors.items() if c == 1}
    return U, V

# Check if graph is bipartite
try:
    U, V = bipartite_colors(G_bip)
    print(f"Graph is bipartite!")
    print(f"Set U: {sorted(U)}")
    print(f"Set V: {sorted(V)}")
except rx.GraphNotBipartite:
    print("Graph is not bipartite")

### Bipartite Projection

Two target nodes are connected in the projection if they share a common neighbor in the original graph.

In [ ]:
def bipartite_project(B, target_nodes):
    """
    Project a bipartite graph onto a set of target nodes.
    Two target nodes are connected if they share a common neighbor.
    """
    other_nodes = set(B.node_indexes()).difference(target_nodes)
    
    # Create subgraph with only target nodes
    G, n_map = B.subgraph_with_nodemap(list(target_nodes))
    ix_map = {v: k for k, v in n_map.items()}
    
    # Find projected edges through shared neighbors
    projected_edges = set()
    for k in other_nodes:
        neighbors = list(B.neighbors(k))
        edges = set(combinations(neighbors, 2))
        projected_edges.update(edges)
    
    # Add edges to new graph
    new_edges = [(ix_map[i], ix_map[j]) for i, j in projected_edges 
                 if i in ix_map and j in ix_map]
    G.add_edges_from_no_data(new_edges)
    
    return G, n_map

# Project onto U
H_u, _ = bipartite_project(G_bip, U)
mpl_draw(H_u, node_color=NS_PURPLE, edge_color='gray',
         with_labels=True, node_size=500, font_color='white')
plt.title("Projection onto U")
plt.show()

---
## 2.8 Paths and Distances

### Key Definitions

| Term | Definition |
|------|------------|
| **Path** | A sequence of nodes where each consecutive pair is connected by an edge |
| **Shortest Path (Geodesic)** | The path with minimum number of edges between two nodes |
| **Distance** ($d_{ij}$) | The length of the shortest path between nodes $i$ and $j$ |
| **Diameter** ($d_{max}$) | The longest shortest path in the network |
| **Average Path Length** ($\langle d \rangle$) | The mean distance between all pairs of nodes |
| **Cycle** | A closed path that starts and ends at the same node |
| **Eulerian Path** | A path that traverses each edge exactly once |
| **Hamiltonian Path** | A path that visits each node exactly once |

In [ ]:
# Generate a random graph
N, p = 10, 0.25
G = rx.undirected_gnp_random_graph(N, p, seed=10)
pos = rx.spring_layout(G, seed=2)

mpl_draw(G, node_color=NS_PURPLE, edge_color=NS_GREEN, pos=pos,
         with_labels=True, node_size=500, font_color='white')
plt.title(f"Random Graph (N={N}, p={p})")
plt.show()

### Shortest Paths

In [ ]:
# Find all shortest paths between two nodes
source, target = 7, 4
all_paths = rx.all_shortest_paths(G, source, target)

print(f"All shortest paths from {source} to {target}:")
for path in all_paths:
    print(f"  {list(path)}, Length: {len(path) - 1}")

### Diameter

The diameter is the maximum distance between any pair of nodes.

In [ ]:
# Compute diameter using all-pairs shortest paths
apsp = rx.all_pairs_bellman_ford_shortest_paths(G, edge_cost_fn=lambda x: 1)

path_len_map = defaultdict(list)
for node, mapping in apsp.items():
    for target, path in mapping.items():
        path_len = len(path) - 1
        path_len_map[path_len].append(path)

diameter = max(path_len_map.keys())
print(f"Diameter of G: {diameter}")
print(f"\nExample paths with length = diameter:")
for path in path_len_map[diameter][:3]:
    print(f"  {list(path)}")

### Eulerian Paths

An undirected graph has an Eulerian path if and only if:
1. Either 0 or 2 vertices have odd degree
2. All vertices with non-zero degree are in one connected component

In [ ]:
def has_eulerian_path(G):
    """Check if graph has an Eulerian path."""
    degrees = [G.degree(n) for n in G.node_indexes()]
    odd_count = sum(d % 2 != 0 for d in degrees)
    components = rx.connected_components(G)
    
    return odd_count in [0, 2] and len(components) == 1

# Create the Königsberg bridges graph
G_konig = rx.PyGraph()
G_konig.add_nodes_from(range(4))
edges = [(0, 1), (0, 1), (1, 2), (1, 2), (1, 3), (0, 3), (2, 3)]
G_konig.add_edges_from_no_data(edges)

mpl_draw(G_konig, node_color=NS_PURPLE, edge_color=NS_GREEN,
         with_labels=True, node_size=500, font_color='white')
plt.title("Königsberg Bridges")
plt.show()

degrees = [G_konig.degree(n) for n in G_konig.node_indexes()]
print(f"Node degrees: {degrees}")
print(f"Nodes with odd degree: {sum(d % 2 != 0 for d in degrees)}")
print(f"\n→ Euler proved it's impossible to cross all bridges exactly once!")

### Average Path Length

The average path length characterizes how efficiently information spreads through a network:

$$\langle d \rangle = \frac{1}{N(N-1)} \sum_{\substack{i,j = 1 \\ i \ne j}}^{N} d_{i,j}$$

In [ ]:
def average_path_length(G):
    """Calculate the average path length of a graph."""
    dm = rx.distance_matrix(G, null_value=np.inf)
    finite = dm[np.isfinite(dm)]
    finite = finite[finite > 0]
    return finite.mean() if len(finite) > 0 else 0

# Create a simple connected graph
G_path = rx.PyGraph()
G_path.add_nodes_from(range(5))
G_path.add_edges_from_no_data([(0, 1), (1, 2), (1, 4), (2, 3), (3, 4)])

avg_d = average_path_length(G_path)
print(f"Average path length ⟨d⟩ = {avg_d:.3f}")

print("\nDistance Matrix:")
print(rx.distance_matrix(G_path))

---
## 2.9 Clustering Coefficient

The clustering coefficient captures the degree to which the neighbors of a given node link to each other.

### Key Definitions

| Term | Formula | Description |
|------|---------|-------------|
| **Local Clustering Coefficient** | $C_i = \frac{2 L_i}{k_i (k_i - 1)}$ | Fraction of possible edges between node $i$'s neighbors that actually exist |
| **$L_i$** | — | Number of edges between neighbors of node $i$ |
| **$k_i$** | — | Degree of node $i$ |
| **Average Clustering Coefficient** | $\langle C \rangle = \frac{1}{N} \sum_{i=1}^{N} C_i$ | Mean clustering coefficient across all nodes |

In [ ]:
# Create a graph to demonstrate clustering
G_clust = rx.generators.complete_graph(5)
pos_clust = {
    0: (0, 0.5),
    1: (-1, -1),
    2: (1, -1),
    3: (1, 1),
    4: (-1, 1),
}

node_colors = [NS_PURPLE] + [NS_ORANGE] * 4
mpl_draw(G_clust, pos=pos_clust, node_color=node_colors, edge_color=NS_GREEN,
         with_labels=True, node_size=500, font_color='white', width=2)
plt.title("Complete Graph (K5) - All neighbors connected")
plt.show()

### Local Clustering Coefficient

For a node $i$ with degree $k_i$, the local clustering coefficient measures how close its neighbors are to forming a complete graph (clique).

In [ ]:
def clustering_coefficient(G, n):
    """Calculate the local clustering coefficient for node n."""
    neighbors = G.neighbors(n)
    k = len(neighbors)
    
    if k < 2:
        return 0.0
    
    # Count edges between neighbors
    edges_between_neighbors = sum(
        G.has_edge(u, v) for u, v in combinations(neighbors, 2)
    )
    
    # Maximum possible edges between neighbors
    possible_edges = math.comb(k, 2)
    
    return edges_between_neighbors / possible_edges

# Complete graph: all neighbors are connected
print("Complete graph K5:")
for n in G_clust.node_indexes():
    print(f"  Node {n}: C = {clustering_coefficient(G_clust, n):.3f}")

In [ ]:
# Remove some edges to see different clustering coefficients
G_sparse = rx.generators.complete_graph(5)
G_sparse.remove_edges_from([(1, 2), (4, 2), (3, 1)])

mpl_draw(G_sparse, pos=pos_clust, node_color=node_colors, edge_color=NS_GREEN,
         with_labels=True, node_size=500, font_color='white', width=2)
plt.title("Sparse Graph - Some neighbors not connected")
plt.show()

print("Sparse graph:")
for n in G_sparse.node_indexes():
    print(f"  Node {n}: C = {clustering_coefficient(G_sparse, n):.3f}")

### Average Clustering Coefficient

The average clustering coefficient characterizes the overall tendency of nodes to cluster together.

In [ ]:
def average_clustering_coefficient(G):
    """Calculate the average clustering coefficient of the graph."""
    N = G.num_nodes()
    return sum(clustering_coefficient(G, i) for i in G.node_indexes()) / N

print(f"Complete graph K5: ⟨C⟩ = {average_clustering_coefficient(G_clust):.3f}")
print(f"Sparse graph:      ⟨C⟩ = {average_clustering_coefficient(G_sparse):.3f}")

### Interpretation

- $C_i = 0$: None of node $i$'s neighbors are connected to each other
- $C_i = 1$: All of node $i$'s neighbors are connected (form a clique)
- High $\langle C \rangle$: Network has strong local clustering (common in social networks)
- Low $\langle C \rangle$: Neighbors rarely connect (common in random networks)

---
## Summary

| Section | Key Concepts |
|---------|--------------|
| **2.3** | Degree, average degree, degree distribution |
| **2.4** | Adjacency matrix, degree from matrix, trace tricks, counting triangles |
| **2.7** | Bipartite graphs and projections |
| **2.8** | Paths, shortest paths, diameter, cycles, Eulerian/Hamiltonian paths, average path length |
| **2.9** | Local clustering coefficient, average clustering coefficient |

---

*Companion notebook for [Network Science](http://networksciencebook.com/) by Albert-László Barabási*